In [60]:
import pandas as pd
import numpy as np
from src.func_model_api import *
from src.func_optimiser_api import *
from src.func_optimisation import *
#from playground.tyler.src.utilities import *

pd.options.mode.chained_assignment = None
%matplotlib inline
%load_ext autoreload
%autoreload
%cd C:\Users\tyler\repos\airbnb-forecast

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
C:\Users\tyler\repos\airbnb-forecast


# 1. Optimisation Experiment: Brute Force

### 1.1 Read test data.

In [38]:
df = pd.read_csv('data/test_jun20.csv', low_memory=False)

col_list = [
    # list
    'ID', 'YEAR_MONTH', 'PROPERTY_TYPE', 'ROOM_TYPE', 'ACCOMMODATES'
    , 'BATHROOMS', 'BEDROOMS', 'BEDS', 'BED_TYPE', 'SQUARE_FEET'
    , 'INSTANT_BOOKABLE', 'GUESTS_INCLUDED', 'CANCELLATION_POLICY'
    , 'REQUIRE_GUEST_PROFILE_PICTURE', 'REQUIRE_GUEST_PHONE_VERIFICATION'
    # cal
    , 'AVG_MINIMUM_NIGHTS', 'AVG_MAXIMUM_NIGHTS'
    # host
    , 'HOST_RESPONSE_RATE','HOST_SINCE'
    , 'HOST_ACCEPTANCE_RATE', 'HOST_IS_SUPERHOST', 'HOST_LISTINGS_COUNT'
    , 'HOST_HAS_PROFILE_PIC', 'HOST_IDENTITY_VERIFIED'
    # review
    , 'NUMBER_OF_REVIEWS','FIRST_REVIEW','LAST_REVIEW'
    , 'REVIEW_SCORES_RATING', 'REVIEW_SCORES_ACCURACY'
    , 'REVIEW_SCORES_CLEANLINESS', 'REVIEW_SCORES_CHECKIN'
    , 'REVIEW_SCORES_COMMUNICATION', 'REVIEW_SCORES_LOCATION'
    , 'REVIEW_SCORES_VALUE', 'REVIEWS_PER_MONTH'
    # location
    , 'NEIGHBOURHOOD_CLEANSED', 'IS_LOCATION_EXACT'
    # , 'LATITUDE', 'LONGITUDE'
    # price
    , 'TXN_PRICE', 'SECURITY_DEPOSIT', 'CLEANING_FEE', 'EXTRA_PEOPLE'
    ]

df = df[col_list]
df=df.rename(columns={'TXN_PRICE':'BASE_PRICE'})
df.head()


,ID,YEAR_MONTH,PROPERTY_TYPE,ROOM_TYPE,ACCOMMODATES,BATHROOMS,BEDROOMS,BEDS,BED_TYPE,SQUARE_FEET,...,REVIEW_SCORES_COMMUNICATION,REVIEW_SCORES_LOCATION,REVIEW_SCORES_VALUE,REVIEWS_PER_MONTH,NEIGHBOURHOOD_CLEANSED,IS_LOCATION_EXACT,BASE_PRICE,SECURITY_DEPOSIT,CLEANING_FEE,EXTRA_PEOPLE
0,9835,2020-06,house,private room,2,1.0,1.0,2.0,real bed,NaN,...,10.0,9.0,9.0,0.04,Manningham,0,60.0,NaN,NaN,22.0
1,10803,2020-06,apartment,private room,2,1.0,1.0,2.0,futon,226.0,...,10.0,10.0,10.0,1.61,Moreland,1,31.0,0.0,14.0,15.0
2,12936,2020-06,apartment,entire home/apt,2,1.0,1.0,1.0,real bed,0.0,...,10.0,10.0,9.0,0.35,Port Phillip,1,95.0,450.0,100.0,0.0
3,38271,2020-06,apartment,entire home/apt,5,1.0,3.0,3.0,real bed,NaN,...,10.0,10.0,10.0,1.28,Casey,1,99.0,0.0,0.0,30.0
4,41836,2020-06,house,private room,2,1.0,1.0,1.0,real bed,NaN,...,9.0,9.0,9.0,1.38,Darebin,1,40.0,NaN,10.0,20.0


### 1.2 Create 10 more prices, get demand curves and plot some of the curves

In [ ]:
# add 10 more prices
df_prices = create_multiple_prices(df)
# get demand curves
df_demand = get_demand_curve(df_prices)
# get a dict of demand curve plots
dict_dc = view_demand_curve(df_demand)

In [ ]:
df_demand.ID.unique()

Plot listing id 9835:

In [21]:
dict_dc[9835]

Plot listing id 41836:
So this one is price inelastic?

In [22]:
dict_dc[41836]

# so this one is price inelastic?

Plot listing id 51592:

In [23]:
dict_dc[51592]


### 1.3 Call optimiser_api function to find the best prices

In [66]:
# Optimiser API
df_best, total_time = optimiser_api(df, op_time=True)
df_best

,PRICE,ID,YEAR_MONTH,BOOKED,PRICE_RATE,REVENUE,OBJ
5,90.0,9835,2020-06,8,1.5,720.0,720.0
20,59.0,10803,2020-06,13,1.9,767.0,767.0
32,190.0,12936,2020-06,13,2.0,2470.0,2470.0
43,198.0,38271,2020-06,11,2.0,2178.0,2178.0
54,80.0,41836,2020-06,10,2.0,800.0,800.0
...,...,...,...,...,...,...,...
258027,480.0,43733191,2020-06,11,1.5,5280.0,5280.0
258038,82.0,43744200,2020-06,13,1.5,1066.0,1066.0
258049,210.0,43746933,2020-06,10,1.5,2100.0,2100.0
258060,118.0,43748923,2020-06,10,1.5,1180.0,1180.0


Time used in optimisation, in seconds

In [67]:
total_time

869

### 1.4 Call the diagnose_output function and
show the overall histogram and the ranked list

In [68]:
plot,df_rank=diagnose_output(df_best)

In [69]:
plot

In [73]:
df_rank.head(10)

,ID,PRICE_RATE,RANKING
129656,25542787,2.0,1.0
137554,27469862,2.0,1.0
137576,27473376,2.0,1.0
137587,27478014,2.0,1.0
137598,27480119,2.0,1.0
247252,42894936,2.0,1.0
247241,42894883,2.0,1.0
137642,27494986,2.0,1.0
137653,27495633,2.0,1.0
137686,27497812,2.0,1.0


In [71]:
df_rank.tail(10)

,ID,PRICE_RATE,RANKING
82335,19180745,1.0,23730.0
230434,41278626,1.0,23730.0
84018,19455563,1.0,23730.0
230423,41278626,1.0,23730.0
255371,43377643,1.0,23730.0
219956,40455826,1.0,23730.0
230412,41278626,1.0,23730.0
220000,40456419,1.0,23730.0
235736,41809439,1.0,23730.0
248925,42976784,1.0,23730.0
